In [ ]:
import pandas as pd
import numpy as np
# from yahoo_fin import options
import yfinance as yf
from datetime import datetime, timedelta
from datetime import date
from dateutil.parser import parse # 데이트 형식 자동변환
from copy import copy
# import openpyxl
# from openpyxl.utils.dataframe import dataframe_to_rows
from math import ceil
from more_itertools import locate
import math
import matplotlib.pyplot as plt # 그래프
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import FinanceDataReader as fdr
from tqdm import tqdm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import random

In [ ]:
nasdaq = pd.DataFrame(yf.download(tickers='^IXIC', period='max')[['Adj Close','Volume']]) #3y
nasdaq.rename(columns = {'Adj Close':'price'},inplace=True)

today = nasdaq.index[-1]#.date()
year1 = today - timedelta(days=250)
year20 = today - timedelta(days=6500)
nasdaq

[*********************100%***********************]  1 of 1 completed


,price,Volume
Date,,
1971-02-05,100.000000,0
1971-02-08,100.839996,0
1971-02-09,100.760002,0
1971-02-10,100.690002,0
1971-02-11,101.449997,0
...,...,...
2023-04-28,12226.580078,5331380000
2023-05-01,12212.599609,5168430000
2023-05-02,12080.509766,5501410000


In [ ]:
# # 로그 수익률
# nasdaq['return'] = pd.DataFrame(np.log(nasdaq.price/nasdaq.price.shift(1)) * 100)
# nasdaq

#### T0 시점에서, 성과와 거래량을 어떻게 쓰기좋은 데이터로 변환하는가? 

##### price: log(PXt0/PXt-20)/20, log(PXt0/PXt-40)/40, log(PXt0/PXt-60)/60 for price, 
##### volume: mean(Volt0-Volt-20), mean(Volt0-volT-40), mean(Volt0-Volt-60) for volume

In [ ]:
# today = nasdaq.index[-1]#.date()
day20 = today - timedelta(days=20)
day40 = today - timedelta(days=40)
day60 = today - timedelta(days=60)
day80 = today - timedelta(days=80)
day100 = today - timedelta(days=100)

In [ ]:
nasdaq20_raw=copy(nasdaq)
nasdaq20_raw['vol_20'] = nasdaq['Volume'].rolling(15).mean() # 20일 
nasdaq20_raw['vol_40'] = nasdaq['Volume'].rolling(28).mean() # 40일
nasdaq20_raw['vol_60'] = nasdaq['Volume'].rolling(44).mean() # 60일 60-16=44
nasdaq20_raw['vol_80'] = nasdaq['Volume'].rolling(58).mean() # 80일 80-22=58
nasdaq20_raw['vol_100'] = nasdaq['Volume'].rolling(72).mean() # 100일 100-28=72
nasdaq20_raw

,price,Volume,vol_20,vol_40,vol_60,vol_80,vol_100
Date,,,,,,,
1971-02-05,100.000000,0,NaN,NaN,NaN,NaN,NaN
1971-02-08,100.839996,0,NaN,NaN,NaN,NaN,NaN
1971-02-09,100.760002,0,NaN,NaN,NaN,NaN,NaN
1971-02-10,100.690002,0,NaN,NaN,NaN,NaN,NaN
1971-02-11,101.449997,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-04-28,12226.580078,5331380000,4.876862e+09,4.740103e+09,5.015181e+09,5.019800e+09,5.159632e+09
2023-05-01,12212.599609,5168430000,4.932486e+09,4.752195e+09,5.030509e+09,5.013422e+09,5.160049e+09
2023-05-02,12080.509766,5501410000,4.987449e+09,4.770760e+09,5.034156e+09,5.009649e+09,5.163004e+09


In [ ]:
nasdaq20_p=copy(nasdaq)
nasdaq20_p['p_a20'] = nasdaq['price'].shift(15)
nasdaq20_p['p_a40'] = nasdaq['price'].shift(28)
nasdaq20_p['p_a60'] = nasdaq['price'].shift(44)
nasdaq20_p['p_a80'] = nasdaq['price'].shift(58)
nasdaq20_p['p_a100'] = nasdaq['price'].shift(72)
nasdaq20_p['p_20'] = nasdaq['price'].shift(-15)
nasdaq20_p

,price,Volume,p_a20,p_a40,p_a60,p_a80,p_a100,p_20
Date,,,,,,,,
1971-02-05,100.000000,0,NaN,NaN,NaN,NaN,NaN,101.779999
1971-02-08,100.839996,0,NaN,NaN,NaN,NaN,NaN,101.839996
1971-02-09,100.760002,0,NaN,NaN,NaN,NaN,NaN,102.070000
1971-02-10,100.690002,0,NaN,NaN,NaN,NaN,NaN,102.779999
1971-02-11,101.449997,0,NaN,NaN,NaN,NaN,NaN,103.000000
...,...,...,...,...,...,...,...,...
2023-04-28,12226.580078,5331380000,12087.959961,11675.540039,11394.940430,12006.950195,11079.160156,NaN
2023-05-01,12212.599609,5168430000,12084.360352,11860.110352,11466.980469,11887.450195,11095.110352,NaN
2023-05-02,12080.509766,5501410000,12031.879883,11669.959961,11455.540039,12113.790039,10957.009766,NaN


In [ ]:
# log(PXt0/PXt-20)/20
nasdaq_df = copy(nasdaq)
nasdaq_df
# np.log(nasdaq.price/nasdaq.price.shift(1)) * 100
#2) daily log returns = ln(today price) - ln(previuos day price)
# log_ret = np.log(price).diff(1).dropna()
nasdaq_df['return_20'] = np.log(nasdaq20_p['price']) - np.log(nasdaq20_p['p_a20'])
nasdaq_df['return_40'] = np.log(nasdaq20_p['price']) - np.log(nasdaq20_p['p_a40'])
nasdaq_df['return_60'] = np.log(nasdaq20_p['price']) - np.log(nasdaq20_p['p_a60'])
nasdaq_df['return_80'] = np.log(nasdaq20_p['price']) - np.log(nasdaq20_p['p_a80'])
nasdaq_df['return_100'] = np.log(nasdaq20_p['price']) - np.log(nasdaq20_p['p_a100'])

# nasdaq_df['return_40'] = np.log(nasdaq20_p['p_a40'] / nasdaq20_p['price'])/40 *100
# nasdaq_df['return_60'] = np.log(nasdaq20_p['p_a60'] / nasdaq20_p['price'])/60 *100
# nasdaq_df['return_80'] = np.log(nasdaq20_p['p_a80'] / nasdaq20_p['price'])/80 *100
# nasdaq_df['return_100'] = np.log(nasdaq20_p['p_a100'] / nasdaq20_p['price'])/100 *100
# nasdaq_df['return_a20'] = np.log(nasdaq20_p['price']/ nasdaq20_p['p_20'] )/20 *100
nasdaq_df

,price,Volume,return_20,return_40,return_60,return_80,return_100
Date,,,,,,,
1971-02-05,100.000000,0,NaN,NaN,NaN,NaN,NaN
1971-02-08,100.839996,0,NaN,NaN,NaN,NaN,NaN
1971-02-09,100.760002,0,NaN,NaN,NaN,NaN,NaN
1971-02-10,100.690002,0,NaN,NaN,NaN,NaN,NaN
1971-02-11,101.449997,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-04-28,12226.580078,5331380000,0.011402,0.046116,0.070443,0.018127,0.098546
2023-05-01,12212.599609,5168430000,0.010556,0.029287,0.062997,0.026985,0.095964
2023-05-02,12080.509766,5501410000,0.004034,0.034575,0.053120,-0.002751,0.097614


In [ ]:
nasdaq['return_a20'] = np.log(nasdaq20_p['p_20']) - np.log(nasdaq20_p['price'])
nasdaq

,price,Volume,return_a20
Date,,,
1971-02-05,100.000000,0,0.017643
1971-02-08,100.839996,0,0.009868
1971-02-09,100.760002,0,0.012917
1971-02-10,100.690002,0,0.020544
1971-02-11,101.449997,0,0.015163
...,...,...,...
2023-04-28,12226.580078,5331380000,NaN
2023-05-01,12212.599609,5168430000,NaN
2023-05-02,12080.509766,5501410000,NaN


In [ ]:
nasdaq_df['vol_20'] = nasdaq20_raw['vol_20']
nasdaq_df['vol_40'] = nasdaq20_raw['vol_40']
nasdaq_df['vol_60'] = nasdaq20_raw['vol_60']
nasdaq_df['vol_80'] = nasdaq20_raw['vol_80']
nasdaq_df['vol_100'] = nasdaq20_raw['vol_100']
nasdaq_df



,price,Volume,return_20,return_40,return_60,return_80,return_100,vol_20,vol_40,vol_60,vol_80,vol_100
Date,,,,,,,,,,,,
1971-02-05,100.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-08,100.839996,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-09,100.760002,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-10,100.690002,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-02-11,101.449997,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,12226.580078,5331380000,0.011402,0.046116,0.070443,0.018127,0.098546,4.876862e+09,4.740103e+09,5.015181e+09,5.019800e+09,5.159632e+09
2023-05-01,12212.599609,5168430000,0.010556,0.029287,0.062997,0.026985,0.095964,4.932486e+09,4.752195e+09,5.030509e+09,5.013422e+09,5.160049e+09
2023-05-02,12080.509766,5501410000,0.004034,0.034575,0.053120,-0.002751,0.097614,4.987449e+09,4.770760e+09,5.034156e+09,5.009649e+09,5.163004e+09


In [ ]:
nasdaq_df20 = nasdaq_df[year20:]
nasdaq_df20

,price,Volume,return_20,return_40,return_60,return_80,return_100,vol_20,vol_40,vol_60,vol_80,vol_100
Date,,,,,,,,,,,,
2005-07-18,2144.870117,1307270000,0.043645,0.036903,0.081610,0.094850,0.070272,1.573263e+09,1.621750e+09,1.617739e+09,1.645784e+09,1.674052e+09
2005-07-19,2173.179932,1632670000,0.060696,0.053398,0.085833,0.119999,0.083474,1.585554e+09,1.622826e+09,1.623247e+09,1.645139e+09,1.672472e+09
2005-07-20,2188.570068,1957070000,0.055753,0.052367,0.088028,0.125506,0.080750,1.610412e+09,1.633197e+09,1.633169e+09,1.648464e+09,1.675844e+09
2005-07-21,2178.600098,2034880000,0.051670,0.054521,0.070327,0.134631,0.085860,1.636713e+09,1.654108e+09,1.634919e+09,1.651457e+09,1.683046e+09
2005-07-22,2179.739990,1660360000,0.057976,0.052160,0.064992,0.126021,0.090006,1.631971e+09,1.662259e+09,1.634005e+09,1.644919e+09,1.686906e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,12226.580078,5331380000,0.011402,0.046116,0.070443,0.018127,0.098546,4.876862e+09,4.740103e+09,5.015181e+09,5.019800e+09,5.159632e+09
2023-05-01,12212.599609,5168430000,0.010556,0.029287,0.062997,0.026985,0.095964,4.932486e+09,4.752195e+09,5.030509e+09,5.013422e+09,5.160049e+09
2023-05-02,12080.509766,5501410000,0.004034,0.034575,0.053120,-0.002751,0.097614,4.987449e+09,4.770760e+09,5.034156e+09,5.009649e+09,5.163004e+09


In [ ]:
# import pandas as pd
# import numpy as np

# # 데이터프레임 생성
# df = pd.DataFrame({'price': [10, 12, 15, 14, 16, 18]})

# # 로그 수익률 계산 함수
# def calc_log_return(x, n):
#     return np.log(x.shift(-n) / x)

# # 2일 뒤의 로그 수익률 계산하여 'log_return_2d' 컬럼 추가
# df['log_return_2d'] = calc_log_return(df['price'], 2)

# # 3일 뒤의 로그 수익률 계산하여 'log_return_3d' 컬럼 추가
# df['log_return_3d'] = calc_log_return(df['price'], 3)

# # 결과 확인
# print(df)


#### 유클리디안 거리

In [ ]:
# def euclidean_distance(A, B):   
#     return np.sqrt(np.sum((A-B)**2))

In [ ]:
import pandas as pd

# 데이터 프레임 불러오기
# df = pd.read_csv('data.csv')

# 가장 마지막 행의 데이터 추출
last_row = nasdaq_df20.iloc[-1,:]


In [ ]:
from scipy.spatial import distance

# 가장 마지막 행과 다른 행들 간의 유클리디안 디스턴스 계산
euclidean_distances = nasdaq_df20.apply(lambda row: distance.euclidean(row, last_row), axis=1)

# 결과 출력
print(euclidean_distances)

Date
2005-07-18    8.317145e+09
2005-07-19    8.181009e+09
2005-07-20    8.041387e+09
2005-07-21    7.990110e+09
2005-07-22    8.125071e+09
                  ...     
2023-04-28    6.112037e+08
2023-05-01    4.390188e+08
2023-05-02    7.583084e+08
2023-05-03    1.036865e+09
2023-05-04    0.000000e+00
Length: 4481, dtype: float64


In [ ]:
eucli = euclidean_distances
eucli_sort = eucli.sort_values()
eucli_sort

Date
2023-05-04    0.000000e+00
2022-08-18    1.399087e+08
2022-08-12    1.612735e+08
2022-08-19    1.757817e+08
2022-08-05    2.335295e+08
                  ...     
2005-09-06    8.490158e+09
2005-08-29    8.496002e+09
2005-11-25    8.510625e+09
2012-05-29    8.562632e+09
2005-09-02    8.594993e+09
Length: 4481, dtype: float64

In [ ]:
eucli_sort100 = eucli_sort.head(100)
eucli_sort100

Date
2023-05-04    0.000000e+00
2022-08-18    1.399087e+08
2022-08-12    1.612735e+08
2022-08-19    1.757817e+08
2022-08-05    2.335295e+08
                  ...     
2022-02-08    6.069823e+08
2022-09-29    6.096892e+08
2022-10-04    6.101662e+08
2023-04-28    6.112037e+08
2022-02-15    6.122881e+08
Length: 100, dtype: float64

In [ ]:
eucli_sort100

Date
2023-05-04    0.000000e+00
2022-08-18    1.399087e+08
2022-08-12    1.612735e+08
2022-08-19    1.757817e+08
2022-08-05    2.335295e+08
                  ...     
2022-02-08    6.069823e+08
2022-09-29    6.096892e+08
2022-10-04    6.101662e+08
2023-04-28    6.112037e+08
2022-02-15    6.122881e+08
Length: 100, dtype: float64

In [ ]:
nasdaq_ex = nasdaq[nasdaq.index.isin(eucli_sort100.index)]
nasdaq_ex

,price,Volume,return_a20
Date,,,
2021-12-08,15786.990234,4615390000,-0.002882
2021-12-09,15517.370117,4515600000,0.008189
2021-12-10,15630.599609,4426210000,0.012853
2021-12-13,15413.280273,4573920000,0.013497
2021-12-14,15237.639648,4865990000,-0.009063
...,...,...,...
2023-04-25,11799.160156,4806020000,NaN
2023-04-27,12142.240234,5253710000,NaN
2023-04-28,12226.580078,5331380000,NaN


In [ ]:
print(nasdaq_ex.mean()) # - 0.003
print()
print(nasdaq_ex.std()) # 0.055

price         1.251469e+04
Volume        4.769405e+09
return_a20   -2.212277e-02
dtype: float64

price         1.310408e+03
Volume        2.528960e+08
return_a20    6.437366e-02
dtype: float64


In [ ]:
# plt.hist(weight)

# plt.show()